In [1]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd

sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'sagemakerbucketfinal' # Mention the created S3 bucket name here
print("Using bucket " + bucket)

Using bucket sagemakerbucketfinal


In [2]:
df = pd.read_csv("train.csv")

In [3]:
#DATA Processing

features = list(df.columns)
label = features.pop(-1)
x = df[features]
y = df[label]
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.15, random_state=0)
trainX = pd.DataFrame(X_train)
trainX[label] = y_train
testX = pd.DataFrame(X_test)
testX[label] = y_test

In [4]:
trainX.to_csv("train-V-1.csv",index = False)
testX.to_csv("test-V-1.csv", index = False)

In [5]:
sk_prefix = "sagemaker/mobile_price_classification/sklearncontainer"
trainpath = sess.upload_data(
    path="train-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)

testpath = sess.upload_data(
    path="test-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)
print(trainpath)
print(testpath)

s3://sagemakerbucketfinal/sagemaker/mobile_price_classification/sklearncontainer/train-V-1.csv
s3://sagemakerbucketfinal/sagemaker/mobile_price_classification/sklearncontainer/test-V-1.csv


In [6]:
%%writefile script.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_curve, auc
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO 
import argparse
import joblib
import os
import numpy as np
import pandas as pd
    
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf
    
if __name__ == "__main__":

    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=0)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train-V-1.csv")
    parser.add_argument("--test-file", type=str, default="test-V-1.csv")

    args, _ = parser.parse_known_args()
    
    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))
    
    features = list(train_df.columns)
    label = features.pop(-1)
    
    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    print()
    
    print("Label column is: ",label)
    print()
    
    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (15%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()
    
  
    print("Training RandomForest Model.....")
    print()
    model =  RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state, verbose = 3,n_jobs=-1)
    model.fit(X_train, y_train)
    print()
    

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model,model_path)
    print("Model persisted at " + model_path)
    print()

    
    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)

Writing script.py


In [15]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role="arn:aws:iam::211740653216:role/service-role/AmazonSageMaker-ExecutionRole-20230715T221421",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0,
    },
    use_spot_instances = True,
    max_wait = 7200,
    max_run = 3600
)

In [16]:
# launch training job, with asynchronous call
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=True)
# sklearn_estimator.fit({"train": datapath}, wait=True)

Using provided s3_resource


INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2023-07-15-16-47-52-808


2023-07-15 16:47:56 Starting - Starting the training job...
2023-07-15 16:48:12 Starting - Preparing the instances for training.........
2023-07-15 16:49:53 Downloading - Downloading input data...
2023-07-15 16:50:23 Training - Downloading the training image...
2023-07-15 16:51:04 Uploading - Uploading generated training model2023-07-15 16:50:59,053 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-07-15 16:50:59,057 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-07-15 16:50:59,100 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-07-15 16:50:59,260 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-07-15 16:50:59,272 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-07-15 16:50:59,284 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-07-15 16:50:59,293 sagemaker


2023-07-15 16:51:19 Completed - Training job completed
Training seconds: 86
Billable seconds: 25
Managed Spot Training savings: 70.9%


In [17]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact persisted at " + artifact)


2023-07-15 16:51:19 Starting - Preparing the instances for training
2023-07-15 16:51:19 Downloading - Downloading input data
2023-07-15 16:51:19 Training - Training image download completed. Training in progress.
2023-07-15 16:51:19 Uploading - Uploading generated training model
2023-07-15 16:51:19 Completed - Training job completed
Model artifact persisted at s3://sagemaker-eu-north-1-211740653216/RF-custom-sklearn-2023-07-15-16-47-52-808/output/model.tar.gz


In [22]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name =  model_name,
    model_data=artifact,
    role="arn:aws:iam::211740653216:role/service-role/AmazonSageMaker-ExecutionRole-20230715T221421",
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,
)

In [26]:
endpoint_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name=endpoint_name,
)

EndpointName=Custom-sklearn-model-2023-07-15-17-04-01


INFO:sagemaker:Creating model with name: Custom-sklearn-model-2023-07-15-17-02-35
INFO:sagemaker:Creating endpoint-config with name Custom-sklearn-model-2023-07-15-17-04-01
INFO:sagemaker:Creating endpoint with name Custom-sklearn-model-2023-07-15-17-04-01


----!

In [28]:
print(predictor.predict(testX[features][0:5].values.tolist()))

[3 0 2 1 3]


In [29]:
sm_boto3.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '34365b71-b2ce-41d8-9c4b-c6b0a759e055',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '34365b71-b2ce-41d8-9c4b-c6b0a759e055',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sat, 15 Jul 2023 17:09:19 GMT'},
  'RetryAttempts': 0}}